In [1]:
import requests
import pandas as pd

USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"


# Function to extract fields from a hit
def extract_fields(hit):
    fields = hit["fields"]
    # Ensure id is a single value, not a list
    assert len(fields["id"]) == 1, "Expected exactly one id field"
    fields_dict = {"id": fields["id"][0]}
    # Add all other fields
    for key, value in fields.items():
        if key != "id":
            fields_dict[key] = (
                value[0] if isinstance(value, list) and len(value) == 1 else value
            )
    return fields_dict


# Initialize empty list to store all records
all_records = []

# Pagination parameters
start = 0
step = 48  # Same as size parameter in the search URL
total_found = float("inf")

while start < total_found:
    # Get search results for current page
    # Note: You'll need to implement the actual API call here
    url = f"https://3d.nih.gov/api/search/type:entry%20AND%20submissionstatus:%22Published%22%20AND%20collectionid:33?start={start}&size={step}&sort=created%20desc"
    response = requests.get(url, headers={"User-Agent": USER_AGENT})
    if response.status_code != 200:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        break
    obj = response.json()

    # Process hits from current page
    hits = obj["hits"]["hit"]
    for hit in hits:
        record = extract_fields(hit)
        all_records.append(record)

    start += step
    total_found = obj["hits"]["found"]

# Create DataFrame from all records
df = pd.DataFrame(all_records)


# Display first few rows and shape of the DataFrame
print(f"DataFrame shape: {df.shape}")
print("\nFirst few rows:")
df.head(5)

DataFrame shape: (207, 36)

First few rows:


,id,type,title,lowercasetitle,submissionstatus,collection,collectionid,collectionbadge,description,category,...,threedpxid,paddedentryid,hasdicom,hasdata,hasmesh,hasmedicalimaging,hasbuild,publisheddate,_score,source
0,20088,entry,Common bulrush (Typhya latifolia) pollen grain...,common bulrush (typhya latifolia) pollen grain...,Published,3D Pollen Library,33,/images/collections/badges/pollen_collection_i...,<p>3D reconstructed&nbsp;pollen grain (tetrahe...,Cells and Organelles,...,3DPX-020088,020088,false,false,true,false,false,2025-02-10T14:04:18.044Z,3.856769,NaN
1,20076,entry,Common bulrush (Typhya latifolia) pollen grain...,common bulrush (typhya latifolia) pollen grain...,Published,3D Pollen Library,33,/images/collections/badges/pollen_collection_i...,<p>3D reconstructed&nbsp;pollen grain (T-shape...,Cells and Organelles,...,3DPX-020076,020076,false,false,true,false,false,2025-02-10T14:02:49.12Z,3.856769,NaN
2,20471,entry,Herb-Robert (Geranium robertianum) pollen grain,herb-robert (geranium robertianum) pollen grain,Published,"[3D Pollen Library, Amgueddfa Cymru, National ...","[33, 39]",/images/collections/badges/pollen_collection_i...,<p>3D reconstructed&nbsp;pollen grain from her...,Cells and Organelles,...,3DPX-020471,020471,false,false,true,false,false,2025-02-10T13:57:10.456Z,3.856769,NaN
3,20469,entry,Fog grass (Holcus lanatus) pollen grain,fog grass (holcus lanatus) pollen grain,Published,"[3D Pollen Library, Amgueddfa Cymru, National ...","[33, 39]",/images/collections/badges/pollen_collection_i...,<p>3D reconstructed&nbsp;pollen grain from fog...,Cells and Organelles,...,3DPX-020469,020469,false,false,true,false,true,2025-02-10T13:55:40.965Z,3.856769,NaN
4,20470,entry,Yellow pimpernel (Lysimachia nemorum) pollen g...,yellow pimpernel (lysimachia nemorum) pollen g...,Published,"[3D Pollen Library, Amgueddfa Cymru, National ...","[33, 39]",/images/collections/badges/pollen_collection_i...,<p>3D reconstructed&nbsp;pollen grain from yel...,Cells and Organelles,...,3DPX-020470,020470,false,false,true,false,true,2025-02-10T13:54:07.918Z,3.856769,NaN


In [2]:
# save to csv
df.to_csv("./data/3d_pollen_library.csv", index=False)

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor
import time
import os
import glob
import shutil


def download_model(row, driver, download_path, timeout=30):
    """Downloads a 3D model from the NIH 3D Print Exchange website.

    Args:
        row: A pandas Series or named tuple containing model metadata, must have 'id' field
        driver: Selenium WebDriver instance to control browser
        download_path: Path where downloaded files should be saved
        timeout: Maximum time in seconds to wait for download (default 30)

    Returns:
        None

    Raises:
        TimeoutException: If page elements cannot be found within timeout period
        Exception: For other errors during download process
    """
    # Convert named tuple to dictionary-like access
    row_dict = row._asdict()

    try:
        # Use row_dict instead of row
        url = f"https://3d.nih.gov/entries/{row_dict['id']}"

        # Navigate to the URL
        driver.get(url)

        # Add wait time for page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        # Find and click the Download link
        download_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[text()='Download']"))
        )
        download_link.click()

        # Find and click the STL label
        stl_label = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//label[text()='stl']"))
        )
        stl_label.click()

        # Find and click the download files button
        download_files_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "downloadfilesBtn"))
        )
        download_files_btn.click()

        # Find and click the terms checkbox
        terms_checkbox = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "termsCheckbox"))
        )
        terms_checkbox.click()

        # Find and click the final Download button
        final_download_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[text()='Download']"))
        )
        final_download_btn.click()
        # Wait for an STL file to appear in the temp directory
        stl_file = None
        start_time = time.time()

        while time.time() - start_time < timeout:
            stl_files = glob.glob(os.path.join(download_path, "*.stl"))
            if stl_files:
                stl_file = stl_files[0]
                break
            time.sleep(0.5)

        if not stl_file:
            raise TimeoutError("Download timed out - no STL file found")

        # Create models directory if it doesn't exist
        models_dir = os.path.join("data", "models")
        os.makedirs(models_dir, exist_ok=True)

        # Get original filename and create new name with ID prefix
        original_name = os.path.basename(stl_file)
        new_name = f"{row_dict['id']}_{original_name}"

        # Move and rename the STL file to models directory
        shutil.move(stl_file, os.path.join(models_dir, new_name))

        # print(f"Extracted STL for entry {row_dict['id']}: {row_dict['title']}")
        # print(f"Saved to: {os.path.join(models_dir, stl_files[0])}")

        # Add small delay between requests to be polite to the server
        time.sleep(2)

    except Exception as e:
        print(f"Error processing entry {row_dict['id']}: {str(e)}")
        return row


# Create models directory if it doesn't exist
models_dir = os.path.join("data", "models")
os.makedirs(models_dir, exist_ok=True)

In [4]:
# Function to create and configure Chrome driver
def create_driver(row_dict):
    # Create a unique download path for this thread
    download_path = os.path.join(os.getcwd(), f"temp_{row_dict['id']}")
    os.makedirs(download_path, exist_ok=True)

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option(
        "prefs",
        {
            "download.default_directory": download_path,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
        },
    )
    # Add these options to prevent Chrome from closing immediately
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=chrome_options)
    return driver, download_path

In [5]:
# Test with a single row
first_row = next(df.head(1).itertuples())
driver, download_path = create_driver(first_row._asdict())
try:
    download_model(first_row, driver, download_path)
finally:
    driver.quit()
    try:
        shutil.rmtree(download_path)
    except:
        pass

In [6]:
# Use ThreadPoolExecutor to parallelize downloads
max_workers = 8  # Adjust this number based on your system's capabilities

# Create a driver for each worker thread
drivers = []
for i in range(max_workers):
    driver, download_path = create_driver({"id": f"worker_{i}"})
    drivers.append((driver, download_path))

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all download tasks
    rows_to_retry = list(df.itertuples())
    max_retries = 3
    retry_count = 0
    timeout = 60

    while retry_count <= max_retries and rows_to_retry:
        # Modify download_model to use the driver for its worker index
        futures = []
        for i, row in enumerate(rows_to_retry):
            worker_idx = i % max_workers
            future = executor.submit(
                download_model,
                row,
                drivers[worker_idx][0],
                drivers[worker_idx][1],
                timeout,
            )
            futures.append(future)

        failed_rows = [
            row
            for row, future in zip(rows_to_retry, futures)
            if future.result() is not None
        ]

        if not failed_rows:
            break

        rows_to_retry = failed_rows
        retry_count += 1
        timeout *= 2  # exponential backoff
        if retry_count <= max_retries and failed_rows:
            print(
                f"\nRetrying {len(failed_rows)} failed downloads (attempt {retry_count}/{max_retries}):"
            )


Error processing entry 20852: Message: 

Error processing entry 20861: Download timed out - no STL file found
Error processing entry 20934: [Errno 2] No such file or directory: 'c:\\Users\\super\\Documents\\GitHub\\sequoia\\data-pipeline\\temp_worker_0\\taxus_baccata_NIH3D.stl'
Error processing entry 21114: [Errno 2] No such file or directory: 'c:\\Users\\super\\Documents\\GitHub\\sequoia\\data-pipeline\\temp_worker_4\\prunella_vulgaris_NIH3D.stl'
Error processing entry 21184: [WinError 2] The system cannot find the file specified: 'c:\\Users\\super\\Documents\\GitHub\\sequoia\\data-pipeline\\temp_worker_3\\sedum_acre_NIH3D.stl'
Error processing entry 21279: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\super\\Documents\\GitHub\\sequoia\\data-pipeline\\temp_worker_7\\arrhenatherum_elatius_NIH3D.stl'
Error processing entry 21542: Download timed out - no STL file found
Error processing entry 21541: Download timed out - no STL fi

In [7]:

# Clean up drivers
for driver, download_path in drivers:
    driver.quit()
    try:
        shutil.rmtree(download_path)
    except:
        pass

if failed_rows:
    print(
        f"\nFailed downloads after {max_retries} attempts ({len(failed_rows)} total):"
    )
    for failed_row in failed_rows:
        print(f"- Entry {failed_row}")


Failed downloads after 3 attempts (60 total):
- Entry Pandas(Index=33, id='20861', type='entry', title='Oregano (Origanum vulgare) pollen grain', lowercasetitle='oregano (origanum vulgare) pollen grain', submissionstatus='Published', collection=['3D Pollen Library', 'Amgueddfa Cymru, National Museum Wales, palynology collection'], collectionid=['33', '39'], collectionbadge='/images/collections/badges/pollen_collection_icon.svg', description='<p>3D reconstructed&nbsp;pollen grain from oregano (<em>Origanum vulgare</em>). 3D model created by Anthony J. Hayes, Cardiff University,&nbsp;in collaboration with Heather Pardoe, Amgueddfa Cymru, National Museum Wales, using methodology as described in&nbsp;&nbsp;<a href="https://www.researchgate.net/publication/324390451_Production_of_3D_Printed_Scale_Models_from_Microscope_Volume_Datasets_for_use_in_STEM_Education" rel="noopener noreferrer" target="_blank">Hayes, A.J. (2017) Production of 3D Printed Scale Models from Microscope Volume Datasets